In [1]:
import json
import onnx
import torch
import numpy as np
import pandas as pd
from torch import nn

In [2]:
class ExampleModel(nn.Module):

    def __init__(self):
        super(ExampleModel, self).__init__()

    def forward(self, x1, x2, x3):
        x4 = x1 + x2
        x5 = x2 - x3
        x = torch.cat([x4, x5], dim=0)
        return x

In [3]:
# Create three tensors
x1 = torch.randn(1)
x2 = torch.randn(1)
x3 = torch.randn(1)

# Create the model
model = ExampleModel()

# Run the model
output = model(x1, x2, x3)
print(output)


tensor([-1.3034,  1.5286])


In [4]:
model_name = "example2"
# Set the model to evaluation mode
model.eval()

# Save the model in ONNX format
torch.onnx.export(
    model,
    (x1, x2, x3),
    f"{model_name}.onnx",
    verbose=True,
    input_names=["x_1", "x_2", "x_3"],
    output_names=["y"],
)

# Load the model
onnx_model = onnx.load(f"{model_name}.onnx")

# Check the model
onnx.checker.check_model(onnx_model)

# Add description to the model
onnx_model.graph.doc_string = "Example model with scalar and vector inputs"

# Add metadata to the model
onnx_model.producer_name = "ExampleModel"
onnx_model.producer_version = "0.0.1"
onnx_model.domain = "example"
onnx_model.model_version = 1

# Save the model
onnx.save(onnx_model, f"{model_name}.onnx")


Exported graph: graph(%x_1 : Float(1, strides=[1], requires_grad=0, device=cpu),
      %x_2 : Float(1, strides=[1], requires_grad=0, device=cpu),
      %x_3 : Float(1, strides=[1], requires_grad=0, device=cpu)):
  %/Add_output_0 : Float(1, strides=[1], requires_grad=0, device=cpu) = onnx::Add[onnx_name="/Add"](%x_1, %x_2), scope: __main__.ExampleModel:: # /tmp/ipykernel_20540/3350249101.py:7:0
  %/Sub_output_0 : Float(1, strides=[1], requires_grad=0, device=cpu) = onnx::Sub[onnx_name="/Sub"](%x_2, %x_3), scope: __main__.ExampleModel:: # /tmp/ipykernel_20540/3350249101.py:8:0
  %y : Float(2, strides=[1], requires_grad=0, device=cpu) = onnx::Concat[axis=0, onnx_name="/Concat"](%/Add_output_0, %/Sub_output_0), scope: __main__.ExampleModel:: # /tmp/ipykernel_20540/3350249101.py:9:0
  return (%y)



## Generating model description

Create and save the model description to be provided to ONNX2FMU.

In [5]:
model_description = {
    "name": "example2",
    "description": "The model defines a simple example model with a scalar input and two vector inputs, one with 'local' variability and one with 'continuous' variability.",
    "FMIVersion": "2.0",
    "inputs": [
        {
            "name": "x_1",
            "description": "A scalar input to the model."
        },
        {
            "name": "x_2",
            "description": "A scalar input to the model."
        },
        {
            "name": "x_3",
            "description": "A scalar input to the model."
        }
    ],
    "outputs": [
        {
            "name": "y",
            "description": "The output array."
        }
    ]
}

# Save model description
with open(f"{model_name}Description.json", "w", encoding="utf-8") as f:
    json.dump(model_description, f, indent=4)

## Generating input file and output for testing

In [6]:
time_steps = 100
# Create 10 arrays of random numbers
data = np.random.random((time_steps, 3))
x1 = data[:, 0]
x2 = data[:, 1]
x3 = data[:, 2]

# Do calculations like in the model
out1 = x1 + x2
out2 = x2 - x3

out = np.vstack([out1, out2]).T

# Compare numpy output with model output
x1_tensor = torch.tensor(x1)
x2_tensor = torch.tensor(x2)
x3_tensor = torch.tensor(x3)

y = []

for i in range(time_steps):
    y.append(
        model(
            x1_tensor[i].unsqueeze(-1),
            x2_tensor[i].unsqueeze(-1),
            x3_tensor[i].unsqueeze(-1)
        )
    )

y = torch.stack(y).numpy()

mse = np.sum(np.pow(y - out, 2))

print(f"Mean squared error {mse}")

# Generate time column
time = np.arange(time_steps)

# Initialize input dataset
columns_data = ["x_1_0", "x_2_0", "x_3_0"]
df_data = pd.DataFrame(data, columns=columns_data)
df_data['time'] = time
df_data.set_index('time', inplace=True)
df_data.to_csv('input.csv')

# Initialize out
df_y = pd.DataFrame(y, columns=[f"output_{i}" for i in range(y.shape[1])])
df_y['time'] = time
df_y.set_index('time', inplace=True)
df_y.to_csv('output.csv')


Mean squared error 0.0
